In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    s3_bucket: str
    s3_key: str
    s3_secret_key: str
    object_key: Path
    local_data_file: Path
    local_train_file: Path
    local_val_file: Path
    local_test_file:Path



@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    local_train_file: Path
    local_val_file: Path
    local_test_file: Path
    local_preprocessor_file: Path

In [17]:
import os
import sys
import pandas as pd
from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging

from cnnClassifier.constants import *
from cnnClassifier.utils import pixel_to_image

ImportError: cannot import name 'pixel_to_image' from 'cnnClassifier.utils' (/home/paladin/Downloads/Facial_Impression_Recognition_Calassification/src/cnnClassifier/utils.py)

In [6]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion   
        secret = self.secret.aws_credential

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            s3_bucket = secret.s3_bucket,
            s3_key = secret.s3_key,
            s3_secret_key = secret.s3_secret_key,
            object_key = secret.object_key,
            local_data_file = config.local_data_file,
            local_train_file = config.local_train_file,
            local_val_file = config.local_val_file,
            local_test_file = config.local_test_file

        )

        return data_ingestion_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config_trs = self.config.data_transformation
        config_ing = self.config.data_ingestion         

        create_directories([config_trs.root_dir])
        data_trnsformation_config = DataTransformationConfig(
            root_dir = config_trs.root_dir,
            local_train_file = config_ing.local_train_file,
            local_val_file = config_ing.local_val_file,
            local_test_file = config_ing.local_test_file,
            local_preprocessor_file = config_trs.local_preprocessor_file

        )

        return data_trnsformation_config


In [7]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

from keras.utils import to_categorical
from sklearn.pipeline import Pipeline

In [13]:
from ensure import ensure_annotations
import numpy as np
import pandas as pd
from keras.models import save_model, load_model
from typing import Any
import pickle

#@ensure_annotations
def pixel_to_image(data, img_height:int, img_width:int):
    """
    convert a series of string numbers to a 3D matrix as imege format

    Args:
        data (Dataframe): data to be used for converting
        img_height (int): image height
        img_width (int): image width              
    """
    
    # Three channels with same pixels
    images = np.empty((len(data), img_height, img_width, 3))
    for i, pixel_string in enumerate(data):
        temp = [float(pixel) for pixel in pixel_string.split(' ')]
        temp = np.asarray(temp).reshape(img_height, img_width)
        for j in range(3):
            images[i,:,:,j] = temp
    images/= 255.0        

    return images

#@ensure_annotations
def save_object(path: Path, obj:Any, h5=False):
    """
    save data as pickel or h5 

    Args:
        path (Path): path to .pkl or .h5 file
        obj : object to be saved in the file
        h5 (boolean): if False, then the object is saved as .pkl      
    """
    try:
        dir_path = os.path.dirname(path)
        os.makedirs(dir_path, exist_ok=True)
        if h5:
            save_model(obj, path)
            logging.info(f"model is saved at {path}")
                
        else:
            with open(path, 'wb') as file_obj:
                pickle.dump(obj, file_obj)
            logging.info(f"pickel file saved at {path}")
    
    except Exception as e:
            raise CustomException(e, sys)

In [10]:

from box import ConfigBox

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_transformer_object(self):
        try:            
            face_recognition_pipeline = Pipeline(
                steps=[
                    ('pixel_to_image', 
                    FunctionTransformer(pixel_to_image, kw_args={'img_height':48, 'img_width':48}))
                ]
            )
            return face_recognition_pipeline            
        
        except Exception as e:
            raise CustomException(e, sys)
    
    
    def initiate_data_transformation(self):
        if not os.path.exists(self.config.local_train_file):
            logging.info(f"WARNING: {self.config.local_train_file} does not exist!")
        
        elif not os.path.exists(self.config.local_val_file):
            logging.info(f"WARNING: {self.config.local_val_file} does not exist!")   

        elif not os.path.exists(self.config.local_test_file):
            logging.info(f"WARNING: {self.config.local_test_file} does not exist!")              
        
        else:  
            
            train_df = pd.read_csv(self.config.local_train_file)
            val_df = pd.read_csv(self.config.local_val_file)
            test_df = pd.read_csv(self.config.local_test_file)
            logging.info('Read train, validation and test data completed')

            logging.info("Obtaining preprocessing object")
            preprocessing_obj=self.get_data_transformer_object() 
            pixel_column = 'pixels'
            target_column_name = 'emotion'             

            logging.info(f"Applying preprocessing object on both train and test dataframes")
            input_train_arr = preprocessing_obj.fit_transform(train_df[pixel_column])            
            input_val_arr = preprocessing_obj.fit_transform(val_df[pixel_column])           
            input_test_arr = preprocessing_obj.transform(test_df[pixel_column]) 

            logging.info(f"Changing target variable format to match with keras")
            target_train_arr = to_categorical(train_df[target_column_name], 7)   
            target_val_arr = to_categorical(val_df[target_column_name], 7)  
            target_test_arr = to_categorical(test_df[target_column_name], 7) 
            
            logging.info('Saved preprocessing object')
            save_object(path=self.config.local_preprocessor_file, obj=preprocessing_obj)

            data = {'X_train':input_train_arr, 
                    'y_train':target_train_arr,
                    'X_val':input_val_arr,
                    'y_val': target_val_arr,
                    'X_test':input_test_arr,
                    'y_test':target_test_arr
                    }

            return ConfigBox(data)

In [11]:
try:
    config = configurationManeger()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data = data_transformation.initiate_data_transformation()

except Exception as e:
    raise CustomException(e, sys)

CustomException: Error occured in python script name [/tmp/ipykernel_20363/2611405870.py] line number [5] error message [Argument path of type <class 'str'> to <function save_object at 0x7fb5ecf25c10> does not match annotation type <class 'pathlib.Path'>]